/storage/ice1/shared/d-pace_community/makerspace-datasets/IMAGE/
Imagenet2012

In [2]:
import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# os.environ["TORCH_LOGS"] = "output_code"
# os.environ["TORCH_LOGS"] = "inductor"
# os.environ["TORCHINDUCTOR_TRACE"] = "1"
# os.environ["TORCHINDUCTOR_VERBOSE"] = "1"
# os.environ["TORCHINDUCTOR_DEBUG"] = "1"
# os.environ["TORCHINDUCTOR_DUMP"] = "1"
# os.environ["TORCHINDUCTOR_COMPILE_THREADS"] = "1"
# os.environ["TORCH_COMPILE_DEBUG"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = (
    # "backend:cudaMallocAsync,"
    "expandable_segments:True,"
    # "garbage_collection_threshold:0.6"
)

import sys
sys.path.append('/home/hice1/yyu496/kaggle/CW')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch._functorch.aot_autograd import aot_module, make_boxed_func
from torch.autograd import grad
from torchvision.datasets import ImageFolder
from torch.optim.lr_scheduler import (
    LinearLR,
    CosineAnnealingLR,
    SequentialLR
)
from torch.optim.optimizer import Optimizer

import torch
from torch.profiler import profile, record_function, ProfilerActivity
import torch._dynamo as dynamo
import torch.nn.utils.parametrize as P
from torch.nn.utils.parametrizations import spectral_norm

import ACT6.cpp_extension as cpp_extension

from ACT6.controller import Controller
import ACT6.cuda_graph_utils as cuda_utils
from ACT6.layers import RMSNorm, DOConv2d, DOConv1d, DOLinear
from ACT6.fusion.fused_layers import DOBatchNormReLU2d
from freq_utils import radial_spectrum_2d, wasserstein_1d, kl_div_spectrum
from VT_test import VisionTransformerV2

import timm
from timm.layers import trunc_normal_

import torchvision.models as models
from torchvision.transforms import v2

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.allow_tf32 = True

import random
import numpy as np
import copy


import actnn
# available choices are ["L0", "L1", "L2", "L3", "L4", "L5"]
actnn.set_optimization_level("L3")


from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchmetrics import Accuracy 


import matplotlib.pyplot as plt
import math

/home/hice1/yyu496/.conda/envs/lib/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
batch_size = 512
warmup_epochs = 25
num_epochs = 512
# ============= Data ==================

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

def safe_collate(batch):
    xs, ys = zip(*batch)
    xs = torch.stack([x.contiguous().clone() for x in xs], 0)
    ys = torch.tensor(ys)
    return xs, ys


transform_train = v2.Compose([
    # --- Convert to tensor FIRST (kills PIL early) ---
    v2.ToImage(),                          # handles PIL → Tensor safely
    v2.ToDtype(torch.float32, scale=True),

    # --- Geometry ---
    v2.Resize((224, 224), antialias=True),
    # v2.RandomResizedCrop(224, scale=(0.5, 1.0), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),

    # --- Color ---
    v2.RandomApply([
        v2.ColorJitter(
            brightness=0.2,
            contrast=0.2,
            saturation=0.2,
            hue=0.1
        )
    ], p=0.5),

    # --- RandAugment (vectorized, NO PIL) ---
    v2.RandAugment(num_ops=4, magnitude=9),

    # --- Normalize ---
    v2.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),

    # --- Random Erasing (tensor-based, fast) ---
    v2.RandomErasing(p=0.5, scale=(0.02, 0.33),
                     ratio=(0.3, 3.3), value=0),
])

check_transform = v2.Compose([
    v2.ToImage(),                          
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])


#/home/hice1/yyu496/scratch/data/cifar10_resized/train
# /storage/ice1/shared/d-pace_community/makerspace-datasets/IMAGE/Imagenet2012/train
#/storage/ice1/shared/d-pace_community/makerspace-datasets/IMAGE/CIFAR-10-images
check_set = ImageFolder(
    root="/home/hice1/yyu496/scratch/data/cifar10_resized/train",
    transform=check_transform
)
# check_set = datasets.CIFAR10(root="./data", train=True,
#         download=True, transform=check_transform)

check_loder = DataLoader(
    check_set, batch_size=batch_size, shuffle=True,
    num_workers=15, pin_memory=True, drop_last=True,
    persistent_workers=True, prefetch_factor=3,
    collate_fn=safe_collate
)

# download and prepare
# train_dataset = datasets.CIFAR10(root="./data", train=True,
#                                  download=True, transform=transform_train)
train_dataset = ImageFolder(
    root="/home/hice1/yyu496/scratch/data/cifar10_resized/train",
    transform=transform_train
)

# DataLoader with GPU-friendly settings
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=15, pin_memory=True, drop_last=True,
                          persistent_workers=True, prefetch_factor=3)



val_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),

    v2.Resize((224, 224), antialias=True),

    v2.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

# /storage/ice1/shared/d-pace_community/makerspace-datasets/IMAGE/Imagenet2012/val
# /storage/ice1/shared/d-pace_community/makerspace-datasets/IMAGE/CIFAR-10-images/test
#/home/hice1/yyu496/scratch/data/cifar10_resized/test
# === CIFAR-10 validation dataset (train=False) ===
# val_dataset = datasets.CIFAR10(
#     root="./data",
#     train=False,
#     download=True,
#     transform=val_transform
# )
val_dataset = ImageFolder(
    root="/home/hice1/yyu496/scratch/data/cifar10_resized/test",
    transform=val_transform
)

# === Validation DataLoader ===
val_loader = DataLoader(
    val_dataset,
    batch_size=512,
    shuffle=False,   
    num_workers=15,
    pin_memory=True,
    persistent_workers=True, 
    prefetch_factor=3
)



def mixup_cutmix(x, y, alpha=1.0, p_cutmix=0.5):
    if alpha <= 0:
        return x, y

    lam = torch.distributions.Beta(alpha, alpha).sample().to(x.device)

    b = x.size(0)
    perm = torch.randperm(b, device=x.device)
    x2, y2 = x[perm], y[perm]

    H, W = x.shape[-2:]

    if torch.rand(()) < p_cutmix:
        # ---- CutMix ----
        cx = torch.randint(W, (1,), device=x.device)
        cy = torch.randint(H, (1,), device=x.device)

        w = (W * torch.sqrt(1 - lam)).long()
        h = (H * torch.sqrt(1 - lam)).long()

        x1 = torch.clamp(cx - w // 2, 0, W)
        x2b = torch.clamp(cx + w // 2, 0, W)
        y1 = torch.clamp(cy - h // 2, 0, H)
        y2b = torch.clamp(cy + h // 2, 0, H)

        x[:, :, y1:y2b, x1:x2b] = x2[:, :, y1:y2b, x1:x2b]
        lam = 1 - ((x2b-x1)*(y2b-y1) / (W*H))
    else:
        # ---- Mixup ----
        x = lam * x + (1 - lam) * x2

    return x, (y, y2, lam)

def replace_activation_func(m):
    for name, child in m.named_children():
        if isinstance(child, nn.ReLU):
            setattr(m, name, nn.SiLU(inplace=True))
            # setattr(m, name, nn.ReLU6(inplace=True))
            # setattr(m, name, nn.LeakyReLU(inplace=True))
            # setattr(m, name, nn.GELU())
        else:
            replace_activation_func(child)


def replace_norm(m):
    for name, child in m.named_children():
        if isinstance(child, nn.LayerNorm):
            setattr(m, name, RMSNorm(dims=child.normalized_shape[-1]))
        else:
            replace_activation_func(child)


def disable_act_inplace(m):
    for module in m.modules():
        if isinstance(module, (nn.ReLU, nn.SiLU, nn.GELU, nn.ReLU6)) and hasattr(module, 'inplace'):
            module.inplace = False

def set_seed(SEED):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

def init_fan_out(m):
    if isinstance(m, (nn.Conv2d,)):
        nn.init.kaiming_normal_(
            m.weight,
            mode='fan_out',
            nonlinearity='relu'
        )
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(
            m.weight,
            mode='fan_in',
            nonlinearity='relu'
        )

        if m.bias is not None:
            nn.init.zeros_(m.bias)


def apply_trunc_init_for_act(module, std=0.02):
    """
    Truncated normal init tuned for ACT stability.
    Applies only to ACT-sensitive layers.
    """

    # ---- Convs (ACT-fragile entry + transition layers) ----
    if isinstance(module, nn.Conv2d):
        # stem conv or downsample conv benefit most
        if module.kernel_size[0] > 1 or module.in_channels < module.out_channels:
            trunc_normal_(module.weight, std=std)
        else:
            nn.init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity='relu')

        if module.bias is not None:
            nn.init.zeros_(module.bias)

    # ---- Linear layers (ViT / MLP / projections) ----
    elif isinstance(module, nn.Linear):
        trunc_normal_(module.weight, std=std)
        if module.bias is not None:
            nn.init.zeros_(module.bias)

    # ---- BatchNorm / LayerNorm ----
    elif isinstance(module, (nn.BatchNorm2d, nn.GroupNorm, nn.LayerNorm)):
        nn.init.ones_(module.weight)
        nn.init.zeros_(module.bias)



def init_orthogonal_conv(m, gain=1.0):
    if isinstance(m, nn.Conv2d):
        nn.init.orthogonal_(m.weight, gain=gain)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

    elif isinstance(m, nn.Linear):
        # nn.init.orthogonal_(m.weight, gain=gain)
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)


def init_orthogonal_conv_v2(n, m, gain=1.0):
    if isinstance(m, nn.Conv2d):
        nn.init.orthogonal_(m.weight, gain=gain)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    if n == 'layer2.0.conv1' and isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)
        print('True')

    elif isinstance(m, nn.Linear):
        # nn.init.orthogonal_(m.weight, gain=gain)
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)


In [4]:
act_cache = None
def forward_hook(name):
    def hook(module, input, output):
        global act_cache
        act_cache = output
    return hook

In [5]:
DIVISION = {
    'pool_kernel_size' : 3
}
# DIVISION = None

config = {
    "default_bits": 2,
    'auto_precision': None,
    'DIVISION' : DIVISION,
    "group_size": 256,
    'batch_size' : batch_size,
    'fp8' : False,
    'depth_point_conv' : False,
    'rms_norm' : False
}

num_classes = 10
# vit_base_patch16_224
# efficientnet_b0
# model = timm.create_model('vit_base_patch16_224', pretrained=False, num_classes=10)
# replace_activation_func(model)
# replace_norm(model)

# set_seed(2)
model = VisionTransformerV2(
    image_size=224,
    patch_size=16,
    in_channels=3,
    num_classes=num_classes,
    embed_dim=384,
    depth=8,
    num_head=6,
    mlp_ratio=4.0,
    attn_p=0.0,
    mlp_p=0.0,
    pos_p=0.0
)
# model = models.resnet50(weights=None)
# model = models.resnet18(weights=None)
# model = models.efficientnet_b0(weights=None)
# model = models.mobilenet_v2(weights=None)
# model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
# model.fc = nn.Linear(model.fc.in_features, num_classes)
# replace_activation_func(model)
disable_act_inplace(model)
# model.apply(lambda m: apply_trunc_init_for_act(m, std=0.02))
# model.apply(init_fan_out)
# model.apply(lambda m: init_orthogonal_conv(m, gain=1.0))
# init(model)
# model.load_state_dict(torch.load("/home/hice1/yyu496/scratch/Model_Checkpoint/ResNet18_bs_8192"))


# model = actnn.QModule(model)
# model.cuda()
# model.compile(fullgraph=True)


# set_seed(2)
# torch.set_rng_state(cached_rng_state['CPU'])
# torch.cuda.set_rng_state(cached_rng_state['GPU'])
# model.load_state_dict(baseline)
# disable_act_inplace(model)
# # init(model)
# for n, m in model.named_modules():
#     if isinstance(m, (nn.Conv1d, nn.Conv2d, nn.Linear)):
#             cahced_init(m.weight)



criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

compute_stream = torch.cuda.Stream()
controller = Controller(model, config, train_loader, criterion, test=False)
controller.iterate(criterion)
controller.warp_model(graph_mode=True, quantizer=True)
# controller.warmup(train_loader, criterion, compute_stream)

# for name, module in controller.traced_model.named_modules():
#     if isinstance(module, (DOConv2d, DOConv1d, DOLinear)):
#         if 'features.8.0' in name:
#             m = module.register_forward_hook(forward_hook(name))
#             print(module)

In [6]:
class SGD_LARS(Optimizer):
    def __init__(self,
                 params,
                 lr=0.1,
                 momentum=0.9,
                 weight_decay=1e-4,
                 eta=0.1,
                 eps=1e-9,
                 nesterov=False,
                 trust_coef=True):
        defaults = dict(lr=lr,
                        momentum=momentum,
                        weight_decay=weight_decay,
                        eta=eta,
                        eps=eps,
                        nesterov=nesterov,
                        trust_coef=trust_coef)
        super().__init__(params, defaults)

    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            lr = group['lr']
            momentum = group['momentum']
            weight_decay = group['weight_decay']
            eta = group['eta']
            eps = group['eps']
            nesterov = group['nesterov']

            for p in group['params']:
                if p.grad is None:
                    continue

                grad = p.grad

                # ---- decoupled weight decay ----
                if weight_decay != 0 and p.ndim > 1:
                    grad = grad.add(p, alpha=weight_decay)

                # ---- LARS trust ratio (only apply to weight tensors) ----
                if p.ndim > 1:      # don't LARS biases / norms
                    w_norm = p.norm()
                    g_norm = grad.norm()
                    trust_ratio = 0

                    if not torch.isfinite(grad).all():
                        print("grad has NaN/Inf")
                    if not torch.isfinite(w_norm):
                        print("w_norm NaN/Inf:", w_norm.item())
                    if not torch.isfinite(g_norm):
                        print("g_norm NaN/Inf:", g_norm.item())

                    if w_norm > 0 and g_norm > 0:
                        trust_ratio = eta * (w_norm / (g_norm + eps))
                    else:
                        trust_ratio = 1.0

                    if not torch.isfinite(trust_ratio):
                        print("trust_ratio NaN/Inf:", trust_ratio.item())
                    print("trust_ratio", trust_ratio.item(), "dw_norm", g_norm.item(), "w_norm", w_norm.item())

                    
                else:
                    trust_ratio = 1.0

                scaled_lr = lr * trust_ratio

                # ---- Momentum buffer ----
                param_state = self.state[p]
                if 'momentum_buffer' not in param_state:
                    buf = param_state['momentum_buffer'] = torch.zeros_like(p)
                else:
                    buf = param_state['momentum_buffer']

                buf.mul_(momentum).add_(grad)  # standard momentum update

                if nesterov:
                    update = grad.add(buf, alpha=momentum)
                else:
                    update = buf

                # ---- Update weights ----
                p.add_(update, alpha=-scaled_lr)

        



In [7]:
acc = Accuracy(task='multiclass', num_classes=10, average='macro')

# opt = torch.optim.SGD([torch.tensor([0.], device='cuda', requires_grad=True)], lr=1e-2 * 5)

opt = torch.optim.AdamW(controller.traced_model.parameters(), lr=(1e-4 * 8), fused=True, capturable=True)
# opt = SGD_LARS(
#     controller.traced_model.parameters(),
#     lr=1e-3 * 3,                     # LARS needs high LR
#     momentum=0.9,
#     weight_decay=1e-7,          # decoupled weight decay
#     eta=0.1,                  # LARS coefficient (0.001–0.02)
#     nesterov=True
# )
# opt = torch.optim.AdamW(model.parameters(), lr=(1e-4), fused=True, capturable=True)
# opt = SGD_LARS(
#     model.parameters(),
#     lr=1.0,                     # LARS needs high LR
#     momentum=0.9,
#     weight_decay=1e-7,          # decoupled weight decay
#     eta=0.001,                  # LARS coefficient (0.001–0.02)
#     nesterov=False
# )


total_timer_start = torch.cuda.Event(enable_timing=True)
total_timer_end = torch.cuda.Event(enable_timing=True)
total_time = 0.0

e_timer_start = torch.cuda.Event(enable_timing=True)
e_timer_end = torch.cuda.Event(enable_timing=True)
partile_time = 0.0


# scheduler = torch.optim.lr_scheduler.OneCycleLR(
#     opt,
#     1e-3,
#     total_steps= num_epochs * len(train_loader),
#     pct_start=0.1,
#     div_factor=10,
#     final_div_factor=1000
# )

# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
#     opt,
#     T_0=10,
#     T_mult=2,
#     eta_min=1e-5
# )

warmup_sched = LinearLR(
    opt,
    start_factor=0.5,   # lr = lr * start_factor
    end_factor=1.0,    # lr = lr * end_factor = lr
    total_iters=warmup_epochs
)

# 2. Cosine decay
cosine_sched = CosineAnnealingLR(
    opt,
    T_max=num_epochs - warmup_epochs,
    eta_min=1e-5
)

# 3. Combine them
scheduler = SequentialLR(
    opt,
    schedulers=[warmup_sched, cosine_sched],
    milestones=[warmup_epochs]
)

cuda_graph_generator = cuda_utils.Graph(controller.traced_model,
                                        criterion,
                                        opt,
                                        train_loader,
                                        compute_stream,
                                        mode='qdrop',
                                        num_of_graph=1,
                                        device='cuda')


# cuda_graph_generator = cuda_utils.Graph(model,
#                                         criterion,
#                                         opt,
#                                         train_loader,
#                                         compute_stream,
#                                         mode='qdrop',
#                                         num_of_graph=1,
#                                         device='cuda')

graphs, compute_stream, static_x, static_y, logits, loss = cuda_graph_generator.capture_cuda_graph_qdrop()

total_graphs = cuda_graph_generator.num_of_graph

In [8]:
@torch.no_grad()
def lowpass_param_2d(t: torch.Tensor, k: int = 3):
    """
    Low-pass filter for parameter tensors that have 2D structure in the last two dims
    (e.g. conv weights [out_c, in_c, kH, kW] or activation-like grads [B, C, H, W]).

    For non-4D tensors or very small spatial dims, returns t unchanged.
    """
    if t.ndim != 4:
        return t

    *prefix, H, W = t.shape
    if H < k or W < k:
        return t  # nothing to do, kernel larger than spatial size

    # Merge prefix dims into batch*channels, so shape -> [N, 1, H, W]
    t_flat = t.view(-1, 1, H, W)
    t_l = F.avg_pool2d(t_flat, kernel_size=k, stride=1, padding=k // 2)
    t_l = F.interpolate(t_l, size=(H, W), mode="bilinear", align_corners=False)

    return t_l.view_as(t)


@torch.no_grad()
def inject_grad_noise_large_batch(
    model,
    step,
    batch_size,
    total_samples,
    base_std=1,
    gamma=0.55,
    lp_kernel: int = 3
):
    # Large-batch SGD noise recovery term
    gns_scale = (1.0 / batch_size - 1.0 / total_samples) ** 0.5
    decay = 1.0 / ((1 + step) ** gamma)

    for p in model.parameters():
        if p.grad is None:
            continue

        # Skip bias / norm (critical for ViT & BN stability)
        if p.ndim == 1:
            continue


        grad_std = p.grad.std().clamp(min=1e-6)

        noise = torch.randn_like(p.grad) * base_std * gns_scale * grad_std * decay

        if lp_kernel is not None and lp_kernel > 1:
            noise = lowpass_param_2d(noise, k=lp_kernel)

        p.grad.add_(noise)


def update_lr(model, scheduler):
    lr = scheduler.get_last_lr()[0]
    for m in model.modules():
        if hasattr(m, 'ema_grad_meta'):
            m.ema_grad_meta['lr'].copy_(lr)


def checkpoint_save_helper(model, acc, max_acc, opt):
    if acc >= max_acc:
        max_acc = acc
        torch.save(model.state_dict(), "/home/hice1/yyu496/scratch/Model_Checkpoint/ResNet18_bs_8192_2.pth")
        torch.save(opt.state_dict(), '/home/hice1/yyu496/scratch/Model_Checkpoint/ResNet18_bs_8192_opt_2.pth')
        return max_acc
    return max_acc

In [ ]:
torch.cuda.empty_cache()

G_GNS = []
G_SNR = []


def gradient_noise_scale(grad):
    tr_sigma = grad.var(0, unbiased=False).sum()
    return tr_sigma / (grad.mean(0).pow(2).sum() + 1e-12)

def signal_to_noise_ratio(grad):
    mu = grad.mean(0)
    tr_sigma = grad.var(0, unbiased=False).sum()
    return mu.norm() / (tr_sigma.sqrt() + 1e-12)

max_acc = 0.0
torch.cuda.synchronize()
total_timer_start.record()
for i in range(num_epochs):
    train_logits = []
    train_y = []
    

    val_logits = []
    val_y = []
    partile_time = 0
    controller.traced_model.train()
    # model.train()
    # aot_model.train()
    G = []
    torch.cuda.synchronize()
    # k = i % total_graphs
    for step, (x, y) in enumerate(train_loader):
        
        # torch.cuda.current_stream().wait_stream(warmup)  
        x, y = x.to('cuda', non_blocking=True), y.to('cuda', non_blocking=True)
        
        e_timer_start.record()
        # compute_stream.wait_stream(torch.cuda.current_stream())
        # with torch.cuda.stream(compute_stream):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16, enabled=True):
                # static_x.copy_(x)
                # static_y.copy_(y)
                # opt.zero_grad(set_to_none=False)

                # total_steps = i * len(train_loader) + step
                # k = step % total_graphs
                # add_smoothout_noise(controller.traced_model)
                # assign_theta(controller.traced_model)
                # graphs[0].replay()
                # remove_smoothout_noise(controller.traced_model)
                # opt.step()
                # scheduler.step()   
                
                # train_logits.append(logits.detach().cpu())
                # train_y.append(static_y.detach().cpu())  

                # for m in controller.traced_model.modules():
                #     if hasattr(m, "qdrop"):
                #         temp = torch.rand(1).item()
                #         m.qdrop.copy_(temp)

            opt.zero_grad()
            raw_logits = controller.traced_model(x)
            # raw_logits = model(x)

            if isinstance(raw_logits, tuple):
                logits = raw_logits[0]
                freq_maps = raw_logits[1]
            else:
                logits = raw_logits
                freq_maps = None

            CE_loss = criterion(logits, y)

            model_spectrum1 = radial_spectrum_2d(freq_maps, num_bins=16)
            target_spectrum = radial_spectrum_2d(x, num_bins=16)
            freq_loss1 = wasserstein_1d(model_spectrum1, target_spectrum)
            # model_spectrum = radial_spectrum_2d(act)
            # target_spectrum = radial_spectrum_2d(x).detach()
            # freq_loss = wasserstein_1d(model_spectrum, target_spectrum)
            # freq_loss = kl_div_spectrum(target_spectrum, model_spectrum)
            # freq_loss = temp_kl(target_spectrum, model_spectrum)
            # freq_loss = vit_spectral_slope_loss(act)
            # freq_loss = local_freq_kl_loss(act, x)
            

            loss = CE_loss + freq_loss1 * 1.0
            # loss = CE_loss

            # inject_grad_noise_large_batch(controller.traced_model, step, batch_size, len(train_loader))
            # inject_grad_noise_large_batch(model, step, batch_size, len(train_loader))
            loss.backward()
            opt.step()
            # scheduler.step()     
            
            # if step % 20 == 0:
            #     temp = torch.nn.utils.parameters_to_vector(p.reshape(-1) for p in controller.traced_model.parameters() if p.grad is not None)
            #     # temp = torch.nn.utils.parameters_to_vector(p.reshape(-1) for p in model.parameters() if p.grad is not None)
            #     G.append(temp)     
                
            e_timer_end.record()
            torch.cuda.synchronize()
            partile_time += e_timer_start.elapsed_time(e_timer_end)

            train_logits.append(logits.detach().cpu())
            train_y.append(y.detach().cpu())

                # print("train logits mean:", logits.mean().item())
                # print("train logits std:", logits.std().item())

    # partile_time = e_timer_start.elapsed_time(e_timer_end)
    # total_time += e_timer_start.elapsed_time(e_timer_end)

    train_logits = torch.cat(train_logits)
    train_y = torch.cat(train_y)
    computed_acc = acc(train_logits, train_y)
    throughtout = len(train_dataset) / (partile_time / 1000)

    # temp_G = torch.cat(G)
    # GNS = gradient_noise_scale(temp_G)
    # SNR = signal_to_noise_ratio(temp_G)

    print(f'Epoch: {i}')
    print(f"Train Loss: {loss}")
    print(f"Learning Rate: {scheduler.get_last_lr()[0]}")
    print(f"Train Accuracy: {computed_acc}")
    print(f'Peak Mem Reserved: {torch.cuda.max_memory_reserved()}')
    print(f'Peak Mem Allocated: {torch.cuda.max_memory_allocated()}')
    print(f'Current train time: {partile_time / 1000} s')
    print(f"Throughout: {throughtout} samples per second")
    # for name, m in controller.traced_model.named_modules():
    #     if isinstance(m, DOBatchNormReLU2d):
    #         if not torch.isfinite(m.running_mean).all() or not torch.isfinite(m.running_var).all():
    #             print("BAD BN:", name)
    #             print("mean finite:", torch.isfinite(m.running_mean).all().item(),
    #                 "var finite:", torch.isfinite(m.running_var).all().item(),
    #                 "var min:", m.running_var.min().item())
    #             break
    # print(f'GNS: {GNS}')
    # print(f'SNR: {SNR}')
    # G_GNS.append(GNS)
    # G_SNR.append(SNR)
    scheduler.step()
    # update_lr(controller.traced_model, scheduler)

    train_logits = []
    train_y = []

    controller.traced_model.eval()
    # model.eval()
    # aot_model.eval()
    with torch.compiler.set_stance("force_eager"):
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to('cuda'), y_val.to('cuda')

                with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16, enabled=True):
                    raw_y_preds  = controller.traced_model(x_val)
                    # raw_y_preds = model(x_val)

                    if isinstance(raw_y_preds, tuple):
                        y_preds = raw_y_preds[0]
                    else:
                        y_preds = raw_y_preds
                val_loss = F.cross_entropy(y_preds, y_val)

                val_logits.append(y_preds.detach().cpu())
                val_y.append(y_val.detach().cpu())

    val_logits = torch.cat(val_logits)
    val_y = torch.cat(val_y)

    valid_computed_acc = acc(val_logits, val_y)
    # max_acc = checkpoint_save_helper(controller.traced_model, valid_computed_acc, max_acc, opt)
    print(f"Val Loss: {val_loss}")
    print(f"Val Accuracy: {valid_computed_acc}\n\n")


total_timer_end.record()
torch.cuda.synchronize()


full_time = total_timer_start.elapsed_time(total_timer_end)

Epoch: 0
Train Loss: 2.0998802185058594
Learning Rate: 0.0004
Train Accuracy: 0.19668948650360107
Peak Mem Reserved: 9412018176
Peak Mem Allocated: 8862983168
Current train time: 53.52324494934082 s
Throughout: 934.1735548232261 samples per second
Val Loss: 1.8671875
Val Accuracy: 0.328900009393692


Epoch: 1
Train Loss: 1.9191701412200928
Learning Rate: 0.00041600000000000003
Train Accuracy: 0.29376572370529175
Peak Mem Reserved: 9684647936
Peak Mem Allocated: 9109489152
Current train time: 37.300074081420895 s
Throughout: 1340.4799114033106 samples per second
Val Loss: 1.6015625
Val Accuracy: 0.42499998211860657


Epoch: 2
Train Loss: 1.7935917377471924
Learning Rate: 0.0004320000000000001
Train Accuracy: 0.3921717703342438
Peak Mem Reserved: 9684647936
Peak Mem Allocated: 9109489152
Current train time: 37.652419174194335 s
Throughout: 1327.9359227538896 samples per second
Val Loss: 1.0859375
Val Accuracy: 0.5252000689506531


Epoch: 3
Train Loss: 1.6243280172348022
Learning Rate: 0.

/home/hice1/yyu496/.conda/envs/lib/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:209: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Val Loss: 0.376953125
Val Accuracy: 0.8284000158309937


Epoch: 25
Train Loss: 1.1225162744522095
Learning Rate: 0.0008
Train Accuracy: 0.7581474781036377
Peak Mem Reserved: 9810477056
Peak Mem Allocated: 9109489152
Current train time: 39.49858935546875 s
Throughout: 1265.8679921458329 samples per second
Val Loss: 0.365234375
Val Accuracy: 0.8249000310897827


Epoch: 26
Train Loss: 1.0700345039367676
Learning Rate: 0.0007999917812188376
Train Accuracy: 0.7645065784454346
Peak Mem Reserved: 9810477056
Peak Mem Allocated: 9109489152
Current train time: 36.9607936706543 s
Throughout: 1352.784803419912 samples per second
Val Loss: 0.310546875
Val Accuracy: 0.8080000281333923


Epoch: 27
Train Loss: 1.048949956893921
Learning Rate: 0.0007999671252173672
Train Accuracy: 0.770566463470459
Peak Mem Reserved: 9810477056
Peak Mem Allocated: 9109489152
Current train time: 37.90036756896973 s
Throughout: 1319.2484191350334 samples per second
Val Loss: 0.232421875
Val Accuracy: 0.8294999599456787



In [ ]:
x, y = next(iter(check_loder))
x, y = x.to('cuda'), y.to('cuda')

In [ ]:
set_seed(42)
# torch.set_rng_state(cached_rng_state['CPU'])
# torch.cuda.set_rng_state(cached_rng_state['GPU'])
num_classes = 10
# model = models.resnet18(weights=None)
# model.fc = nn.Linear(model.fc.in_features, num_classes)
# model = models.efficientnet_b0(weights=None)
# model = models.efficientnet_v2_s(weights=None)
model = models.mobilenet_v2(weights=None)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
# model.apply(lambda m: apply_trunc_init_for_act(m, std=0.02))
# model.apply(init_fan_out)
# model.apply(lambda m: init_orthogonal_conv(m, gain=math.sqrt(2)))  # ReLU-friendly
# model.apply(lambda m: init_orthogonal_conv(m, gain=1.0))           # smoother spectrum
# model = timm.create_model('vit_base_patch16_224', pretrained=False, num_classes=10)

disable_act_inplace(model)
# model.load_state_dict(baseline)
# init(model)
# for n, m in model.named_modules():
#     if isinstance(m, (nn.Conv1d, nn.Conv2d)):
#             cahced_init(m.weight)

# for n, m in model.named_modules():
#     init_orthogonal_conv(n, m, gain=1.0)

# for m in model.modules():
#     if isinstance(m, nn.Conv2d) and m.kernel_size == (1, 1):
#         spectral_norm(m)

model = model.to('cuda')


criterion = nn.CrossEntropyLoss()

In [ ]:
model.load_state_dict(torch.load("/home/hice1/yyu496/scratch/Model_Checkpoint/ResNet18_bs_8192_acc_89"))

<All keys matched successfully>

In [ ]:
activation_cache.clear()
torch.cuda.empty_cache()

In [ ]:
model.eval()
activation_cache = {}
low_frequency_energy_ratio_grad = {}
low_frequency_energy_ratio_act = {}


def forward_hook(name):
    def hook(module, input, output):
        activation_cache[name] = output.detach().cpu()
    return hook

@torch.no_grad()
def compute_lfer_act(x, k=4):
    if x.ndim == 4:
        H, W = x.shape[-2:]
        k = min(k, H, W)
        x_lp = F.interpolate(F.avg_pool2d(x, kernel_size=k, stride=k, padding=0), size=x.shape[-2:], mode='nearest')
        return (x_lp.square().sum() / (x.square().sum() + 1e-7)).item()
    else:
        feature_dim = x.shape[-1]
        k = min(k, feature_dim)
        if x.ndim != 2:
            pooled = F.avg_pool1d(x, kernel_size=k, stride=k, padding=0)
            x_lp = F.interpolate(pooled, size=feature_dim, mode='nearest')
        else:
            x_lp = x.mean(dim=-1, keepdim=True).expand_as(x)
        return (x_lp.square().sum() / (x.square().sum() + 1e-7)).item().detach().cpu()


for name, module in model.named_modules():
    if isinstance(module, (nn.Conv1d, nn.Conv2d)):
        module.register_forward_hook(forward_hook(name))

with torch.no_grad():
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        logits = model(x)
        loss = criterion(logits, y)



In [ ]:
activation_cache.keys()

dict_keys(['conv1', 'bn1', 'relu', 'maxpool', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1', 'layer1', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.downsample.0', 'layer2.0.downsample.1', 'layer2.0.downsample', 'layer2.0', 'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu', 'layer2.1.conv2', 'layer2.1.bn2', 'layer2.1', 'layer2', 'layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.relu', 'layer3.0.conv2', 'layer3.0.bn2', 'layer3.0.downsample.0', 'layer3.0.downsample.1', 'layer3.0.downsample', 'layer3.0', 'layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.relu', 'layer3.1.conv2', 'layer3.1.bn2', 'layer3.1', 'layer3', 'layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.relu', 'layer4.0.conv2', 'layer4.0.bn2', 'layer4.0.downsample.0', 'layer4.0.downsample.1', 'layer4.0.downsample', 'layer4.0', 'layer4.1.conv1', 'layer4.

In [ ]:
def radial_spectrum_2d_per_sample(x, num_bins=64, eps=1e-8, out_dtype=None,
                                  *, vit_grid=None, num_prefix_tokens=None, max_prefix_tokens=16,
                                  log_power=True):
    if out_dtype is None:
        out_dtype = x.dtype

    # --- your ViT token -> [B,C,H,W] conversion stays the same ---
    if x.dim() == 3:
        import math
        B, N, D = x.shape
        if vit_grid is not None:
            H, W = vit_grid
            needed = H * W
            if num_prefix_tokens is None:
                num_prefix_tokens = N - needed
            if N - num_prefix_tokens != needed:
                raise ValueError("grid mismatch")
        else:
            if num_prefix_tokens is None:
                found = None
                for k in range(0, max_prefix_tokens + 1):
                    m = N - k
                    if m <= 0: break
                    s = int(math.isqrt(m))
                    if s * s == m:
                        found = k
                        H = W = s
                        break
                if found is None:
                    raise ValueError("cannot infer grid")
                num_prefix_tokens = found
            else:
                m = N - num_prefix_tokens
                s = int(math.isqrt(m))
                if s * s != m:
                    raise ValueError("not square")
                H = W = s

        if num_prefix_tokens > 0:
            x = x[:, num_prefix_tokens:, :].contiguous()
        x = x.view(B, H, W, D).permute(0, 3, 1, 2).contiguous()

    elif x.dim() != 4:
        raise ValueError(f"Expected dim 3 or 4, got {x.dim()}")

    # x: [B,C,H,W]
    x = x.mean(dim=1)  # [B,H,W]

    x_fft = x.to(torch.float32)
    F = torch.fft.fft2(x_fft)
    F = torch.fft.fftshift(F, dim=(-2, -1))
    P = (F.real * F.real + F.imag * F.imag)  # [B,H,W]
    if log_power:
        P = torch.log1p(P)

    B, H, W = P.shape
    yy, xx = torch.meshgrid(torch.arange(H, device=P.device),
                            torch.arange(W, device=P.device), indexing="ij")
    cy, cx = (H - 1) / 2.0, (W - 1) / 2.0
    r = torch.sqrt((yy - cy) ** 2 + (xx - cx) ** 2)
    r = (r / (r.max() + 1e-12) * (num_bins - 1)).to(torch.long)  # [H,W]
    r_flat = r.flatten()  # [HW]

    vals = P.reshape(B, -1)  # [B,HW]

    # per-sample scatter_add
    spec = torch.zeros((B, num_bins), device=P.device, dtype=torch.float32)
    spec.scatter_add_(1, r_flat[None, :].expand(B, -1), vals)

    spec = spec / (spec.sum(dim=1, keepdim=True) + eps)
    return spec.to(out_dtype)


def vit_spectral_slope_loss(tokens, num_bins=64, alpha=2.0, num_prefix_tokens=1):
    spec = radial_spectrum_2d_per_sample(
        tokens, num_bins=num_bins,
        num_prefix_tokens=num_prefix_tokens,
        log_power=True
    )  # [B,K]

    K = spec.shape[1]
    k = torch.arange(K, device=spec.device, dtype=spec.dtype)
    target = 1.0 / (k + 1.0) ** alpha
    target = target / target.sum()

    # compare each sample spectrum to 1/f^alpha
    return torch.mean(torch.sum(torch.abs(spec - target[None, :]), dim=1))

In [ ]:
import math
import torch
import torch.nn.functional as F


def _infer_vit_grid(N, num_prefix_tokens=None, max_prefix_tokens=16):
    if num_prefix_tokens is not None:
        M = N - num_prefix_tokens
        s = int(math.isqrt(M))
        if s * s != M:
            raise ValueError(f"N - num_prefix_tokens = {M} not square")
        return s, s, num_prefix_tokens

    # auto infer
    for k in range(0, max_prefix_tokens + 1):
        M = N - k
        if M <= 0:
            break
        s = int(math.isqrt(M))
        if s * s == M:
            return s, s, k
    raise ValueError(f"Cannot infer vit grid from N={N}")


def _radial_bins(H, W, num_bins, device):
    yy, xx = torch.meshgrid(
        torch.arange(H, device=device),
        torch.arange(W, device=device),
        indexing="ij"
    )
    cy, cx = (H - 1) / 2.0, (W - 1) / 2.0
    r = torch.sqrt((yy - cy) ** 2 + (xx - cx) ** 2)
    r = (r / (r.max() + 1e-12) * (num_bins - 1)).to(torch.long)  # [H,W]
    return r


def local_radial_spectrum_2d(
    x,                      # [B,1,H,W] or [B,H,W]
    window=8,
    num_bins=32,
    eps=1e-8,
):
    """
    Returns local spectra per window:
      spectra: [B, nwin, num_bins] each normalized to sum=1
    """
    if x.dim() == 4:
        x = x.squeeze(1)
    assert x.dim() == 3
    B, H, W = x.shape

    # crop to divisible by window
    Hc = (H // window) * window
    Wc = (W // window) * window
    x = x[:, :Hc, :Wc]

    # unfold into windows: [B, nH, nW, window, window]
    nH = Hc // window
    nW = Wc // window
    xw = x.view(B, nH, window, nW, window).permute(0, 1, 3, 2, 4).contiguous()
    xw = xw.view(B, nH * nW, window, window)  # [B, nwin, w, w]

    # FFT per window (fp32)
    X = xw.to(torch.float32)
    Fw = torch.fft.fft2(X)                      # complex64
    Fw = torch.fft.fftshift(Fw, dim=(-2, -1))
    Pw = (Fw.real * Fw.real + Fw.imag * Fw.imag)  # [B,nwin,w,w]

    # build radial bins for (window, window)
    r = _radial_bins(window, window, num_bins, device=Pw.device)  # [w,w]
    r_flat = r.flatten()  # [w*w]

    # scatter-add radial power
    spectra = torch.zeros((B, nH * nW, num_bins), device=Pw.device, dtype=torch.float32)
    vals = Pw.reshape(B, nH * nW, -1)  # [B,nwin,w*w]

    # vectorized scatter: flatten over B*nwin
    BN = B * (nH * nW)
    spectra2 = torch.zeros((BN, num_bins), device=Pw.device, dtype=torch.float32)
    idx = r_flat[None, :].expand(BN, -1)  # [BN,w*w]
    spectra2.scatter_add_(1, idx, vals.reshape(BN, -1))

    spectra = spectra2.view(B, nH * nW, num_bins)
    spectra = spectra / (spectra.sum(dim=-1, keepdim=True) + eps)

    return spectra  # [B,nwin,K]


def local_freq_kl_loss(
    vit_tokens,             # [B,N,D] tokens (output of some block or final tokens)
    images,                 # [B,3,H_img,W_img] input image tensor
    *,
    vit_grid=None,          # (H,W) patch grid override
    num_prefix_tokens=None, # if known
    max_prefix_tokens=16,
    window=7,
    num_bins=32,
    low_freq_bins=16,       # only match first few bins
    eps=1e-8,
    detach_target=True,
):
    """
    Local spectrum KL loss:
      KL( target || model )

    Returns scalar loss.
    """
    assert vit_tokens.dim() == 3
    B, N, D = vit_tokens.shape

    # infer token grid
    if vit_grid is not None:
        Ht, Wt = vit_grid
        needed = Ht * Wt
        if num_prefix_tokens is None:
            num_prefix_tokens = N - needed
        assert N - num_prefix_tokens == needed
    else:
        Ht, Wt, num_prefix_tokens = _infer_vit_grid(N, num_prefix_tokens, max_prefix_tokens)

    # patch tokens only
    x = vit_tokens[:, num_prefix_tokens:, :]  # [B, Ht*Wt, D]
    x = x.view(B, Ht, Wt, D).permute(0, 3, 1, 2).contiguous()  # [B,D,Ht,Wt]

    # make it 1-channel "activation image"
    x_map = x.mean(dim=1, keepdim=True)  # [B,1,Ht,Wt]

    # downsample input image to patch grid resolution
    img = images
    img_gray = img.mean(dim=1, keepdim=True)  # [B,1,H_img,W_img]
    img_ds = F.interpolate(img_gray, size=(Ht, Wt), mode="bilinear", align_corners=False)

    if detach_target:
        img_ds = img_ds.detach()

    # compute local spectra
    spec_model = local_radial_spectrum_2d(x_map, window=window, num_bins=num_bins, eps=eps)  # [B,nwin,K]
    spec_tgt   = local_radial_spectrum_2d(img_ds, window=window, num_bins=num_bins, eps=eps) # [B,nwin,K]

    # use only low-freq bins
    K = min(low_freq_bins, num_bins)
    pm = spec_model[..., :K].clamp_min(eps)
    pt = spec_tgt[..., :K].clamp_min(eps)

    # renormalize after truncation
    pm = pm / pm.sum(dim=-1, keepdim=True)
    pt = pt / pt.sum(dim=-1, keepdim=True)

    # KL(target || model)
    loss = (pt * (pt.log() - pm.log())).sum(dim=-1)  # [B,nwin]
    return loss.mean()


In [ ]:
import math
import torch

def radial_spectrum_2d(
    x,
    num_bins=64,
    eps=1e-8,
    out_dtype=None,
    *,
    vit_grid=None,              # (H, W) explicit patch grid
    num_prefix_tokens=None,      # number of prefix tokens total
    max_prefix_tokens=16,
    use_cls_token=True,         # NEW: if True, build spectrum from CLS token only
):
    """
    Supports:
      - CNN feature maps: x [B, C, H, W]
      - ViT tokens:       x [B, N, D]  (tokens, embed dim)

    Returns:
      spectrum [num_bins], normalized to sum=1 (aggregated across batch).
    """

    if out_dtype is None:
        out_dtype = x.dtype

    if x.dim() == 3:
        # ViT tokens: [B, N, D]
        B, N, D = x.shape

        # Determine grid size and how many prefix tokens exist.
        if vit_grid is not None:
            H, W = vit_grid
            needed = H * W
            if num_prefix_tokens is None:
                num_prefix_tokens = N - needed
            if N - num_prefix_tokens != needed:
                raise ValueError(
                    f"vit_grid={vit_grid} implies {needed} patch tokens, but got N={N} "
                    f"with num_prefix_tokens={num_prefix_tokens} => {N - num_prefix_tokens} tokens."
                )
        else:
            # Auto-infer num_prefix_tokens by finding N-k that is a perfect square
            if num_prefix_tokens is None:
                found = None
                for k in range(0, max_prefix_tokens + 1):
                    m = N - k
                    if m <= 0:
                        break
                    s = int(math.isqrt(m))
                    if s * s == m:
                        found = k
                        H = W = s
                        break
                if found is None:
                    raise ValueError(
                        f"Cannot infer square token grid from N={N}. "
                        f"Pass vit_grid=(H,W) or num_prefix_tokens explicitly."
                    )
                num_prefix_tokens = found
            else:
                m = N - num_prefix_tokens
                s = int(math.isqrt(m))
                if s * s != m:
                    raise ValueError(
                        f"N - num_prefix_tokens = {m} is not a perfect square. "
                        f"Pass vit_grid=(H,W) instead."
                    )
                H = W = s

        # =========================
        # NEW: CLS-token spectrum
        # =========================
        if use_cls_token:
            # CLS assumed to be token 0
            cls = x[:, 0, :]  # [B, D]

            # Turn CLS embedding into a 2D "image" so FFT makes sense.
            # We reshape D -> H*W (requires D == H*W).
            # If not equal, we crop or pad.
            HW = H * W
            if D < HW:
                pad = HW - D
                cls = torch.nn.functional.pad(cls, (0, pad), mode="constant", value=0.0)
            elif D > HW:
                cls = cls[:, :HW]

            # [B, HW] -> [B, H, W] -> [B, 1, H, W]
            x = cls.view(B, H, W).unsqueeze(1).contiguous()

        else:
            # Default: patch-token spectrum (original behavior)
            if num_prefix_tokens > 0:
                x = x[:, num_prefix_tokens:, :].contiguous()  # [B, H*W, D]
            else:
                x = x.contiguous()

            # [B, H*W, D] -> [B, H, W, D] -> [B, D, H, W]
            x = x.view(B, H, W, D).permute(0, 3, 1, 2).contiguous()

    elif x.dim() != 4:
        raise ValueError(f"Expected x dim 3 or 4, got {x.dim()}")

    # CNN path (or converted ViT path): x [B, C, H, W]
    x = x.mean(dim=1)                 # [B, H, W]

    # FFT in fp32 for safety/stability
    x_fft = x.to(torch.float32)
    F = torch.fft.fft2(x_fft)         # complex64
    F = torch.fft.fftshift(F)  # shift only spatial dims

    P = (F.real * F.real + F.imag * F.imag)  # [B, H, W] float32

    B, H, W = P.shape
    yy, xx = torch.meshgrid(
        torch.arange(H, device=P.device),
        torch.arange(W, device=P.device),
        indexing="ij"
    )
    cy, cx = (H - 1) / 2.0, (W - 1) / 2.0
    r = torch.sqrt((yy - cy) ** 2 + (xx - cx) ** 2)
    r = (r / (r.max() + 1e-12) * (num_bins - 1)).to(torch.long)  # [H, W]

    spectrum = torch.zeros(num_bins, device=P.device, dtype=torch.float32)
    idx = r.flatten().expand(B, -1)     # [B, HW]
    vals = P.reshape(B, -1)            # [B, HW]
    spectrum.scatter_add_(0, idx.reshape(-1), vals.reshape(-1))

    spectrum = spectrum / (spectrum.sum() + eps)
    return spectrum.to(out_dtype)



def wasserstein_1d(p, q, eps=1e-8):
    """
    p, q : tensors [K]
    """
    p32 = p.to(torch.float32)
    q32 = q.to(torch.float32)

    p32 = p32 / (p32.sum() + eps)
    q32 = q32 / (q32.sum() + eps)

    cdf_p = torch.cumsum(p32, dim=0)
    cdf_q = torch.cumsum(q32, dim=0)

    return torch.sum(torch.abs(cdf_p - cdf_q)).to(p.dtype)


def kl_div_spectrum(q, p, eps=1e-8):
    # q, p: [B,K], each row sums to 1
    q = q.clamp_min(eps)
    p = p.clamp_min(eps)
    return (q * (q.log() - p.log())).sum(dim=-1).mean()


def temp_kl(p, q, T=2.0, eps=1e-8):
    """
    p, q: [B, K] distributions (sum=1)
    Applies temperature to log-prob space:
      softmax(log(p)/T)
    """
    logp = torch.log(p.clamp_min(eps))
    logq = torch.log(q.clamp_min(eps))

    pT = F.softmax(logp / T, dim=-1)
    qT = F.softmax(logq / T, dim=-1)

    # KL(p||q) = sum p * (log p - log q)
    kl = (pT * (torch.log(pT.clamp_min(eps)) - torch.log(qT.clamp_min(eps)))).sum(dim=-1).mean()

    return (T * T) * kl


In [ ]:
activation_cache.keys()

NameError: name 'activation_cache' is not defined

In [ ]:
model_spectrum = radial_spectrum_2d(activation_cache['layer4.1.conv2'].to(torch.float32))
target_spectrum = radial_spectrum_2d(x.to(torch.float32)).cpu()

In [ ]:
wasserstein_1d(model_spectrum, target_spectrum)

tensor(5.4471)

In [ ]:
for n, m in model.named_modules():
    if isinstance(m, (nn.Linear, nn.Conv1d, nn.Conv2d)):
        print(n)

features.0.0
features.1.0.block.0.0
features.1.0.block.1.fc1
features.1.0.block.1.fc2
features.1.0.block.2.0
features.2.0.block.0.0
features.2.0.block.1.0
features.2.0.block.2.fc1
features.2.0.block.2.fc2
features.2.0.block.3.0
features.2.1.block.0.0
features.2.1.block.1.0
features.2.1.block.2.fc1
features.2.1.block.2.fc2
features.2.1.block.3.0
features.3.0.block.0.0
features.3.0.block.1.0
features.3.0.block.2.fc1
features.3.0.block.2.fc2
features.3.0.block.3.0
features.3.1.block.0.0
features.3.1.block.1.0
features.3.1.block.2.fc1
features.3.1.block.2.fc2
features.3.1.block.3.0
features.4.0.block.0.0
features.4.0.block.1.0
features.4.0.block.2.fc1
features.4.0.block.2.fc2
features.4.0.block.3.0
features.4.1.block.0.0
features.4.1.block.1.0
features.4.1.block.2.fc1
features.4.1.block.2.fc2
features.4.1.block.3.0
features.4.2.block.0.0
features.4.2.block.1.0
features.4.2.block.2.fc1
features.4.2.block.2.fc2
features.4.2.block.3.0
features.5.0.block.0.0
features.5.0.block.1.0
features.5.0

In [ ]:
model.eval()
activation_cache = {}
low_frequency_energy_ratio_grad = {}
low_frequency_energy_ratio_act = {}
baseline = copy.deepcopy(model.state_dict())

hook_handles = {}
cahced_init = None
cached_rng_state = None

def kaiming_normal_fan_in(w):
    return nn.init.kaiming_normal_(w, mode='fan_in', nonlinearity='relu')
def kaiming_normal_fan_out(w):
    return nn.init.kaiming_normal_(w, mode='fan_out', nonlinearity='relu')
def kaiming_uniform_fan_in(w):
    return nn.init.kaiming_uniform_(w, mode='fan_in', nonlinearity='relu')
def kaiming_uniform_fan_out(w):
    return nn.init.kaiming_uniform_(w, mode='fan_out', nonlinearity='relu')
def trunc_normal(w):
    return trunc_normal_(w, std=0.02)
def trunc_normal_small(w):
    return trunc_normal_(w, std=0.01)
def orthogonal(w):
    return nn.init.orthogonal_(w, gain=1.0)
def orthogonal2(w):
    return nn.init.orthogonal_(w, gain=math.sqrt(2))
def xavier_normal(w):
    return nn.init.xavier_normal_(w)
def xavier_uniform(w):
    return nn.init.xavier_uniform_(w)
def lecun_normal(w):
    return nn.init.normal_(w, std=1.0 / math.sqrt(w.shape[1]))
def kaiming_low_std(w):
    fan_in = nn.init._calculate_correct_fan(w, "fan_in")
    std = math.sqrt(2.0 / fan_in) * 0.5
    return nn.init.normal_(w, std=std)


def forward_hook(name):
    def hook(module, input, output):
        activation_cache[name] = output.detach()
    return hook


min_score = 100
for init in [
    kaiming_normal_fan_in,
    kaiming_normal_fan_out,
    kaiming_uniform_fan_in,
    kaiming_uniform_fan_out,
    kaiming_low_std,
    trunc_normal,
    trunc_normal_small,
    orthogonal,
    orthogonal2,
    xavier_normal,
    xavier_uniform,
    lecun_normal
    ]:

    # 42, 49, 0, 1, 2, 3, 4
    set_seed(2)
    cpu_rng_state = torch.get_rng_state()
    gpu_rng_state = torch.cuda.get_rng_state()

    for m in model.modules():
        if isinstance(m, (nn.Conv1d, nn.Conv2d, nn.Linear)):
            init(m.weight)
    print(init)

    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv1d, nn.Conv2d, nn.Linear)):
            hook_handles[name] = module.register_forward_hook(forward_hook(name))

    with torch.no_grad():
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            logits = model(x)
            loss = criterion(logits, y)

    act = activation_cache['features.18.0']

    model_spectrum = radial_spectrum_2d(act.to(torch.float32)).cpu()
    target_spectrum = radial_spectrum_2d(x.to(torch.float32)).cpu()

    score = wasserstein_1d(model_spectrum, target_spectrum)
    if score < min_score:
        min_score = score
        cahced_init = init
        cached_rng_state = {
            'CPU' : cpu_rng_state,
            'GPU' : gpu_rng_state
        }
        print(f"min score: {min_score}")


    for h in hook_handles.values():
        h.remove()
    activation_cache.clear()
    torch.cuda.empty_cache()



<function kaiming_normal_fan_in at 0x1551dff54670>
min score: 9.649715423583984
<function kaiming_normal_fan_out at 0x1553ec0a1ea0>
min score: 3.960505485534668
<function kaiming_uniform_fan_in at 0x1553ea4969e0>
<function kaiming_uniform_fan_out at 0x1553ec0a1c60>
<function kaiming_low_std at 0x1553ea496950>
min score: 3.7391810417175293
<function trunc_normal at 0x1551dff54820>
<function trunc_normal_small at 0x1553ed6892d0>
<function orthogonal at 0x1553ed689360>
<function orthogonal2 at 0x1553ed689480>
<function xavier_normal at 0x1553e937e200>
<function xavier_uniform at 0x1551dff54700>
<function lecun_normal at 0x1551dff545e0>


In [ ]:
cahced_init

<function __main__.orthogonal2(w)>

In [ ]:
torch.cuda.empty_cache()

In [ ]:
#torch vision default
tensor(17.0655, device='cuda:0')

# timm's trunc_init
tensor(25.9191, device='cuda:0')

# fan out
tensor(11.3065, device='cuda:0')

# init_orthogonal_conv(m, gain=math.sqrt(2))
tensor(22.3052, device='cuda:0')

# init_orthogonal_conv(m, gain=1.0)
tensor(7.9659, device='cuda:0')

NameError: name 'tensor' is not defined

In [ ]:
x, y = next(iter(train_loader))
x, y = x.to('cuda'), y.to('cuda')

In [ ]:
with profile(
    activities=[
        ProfilerActivity.CPU,
        ProfilerActivity.CUDA
    ],
    record_shapes=True,
    with_stack=True,
    profile_memory=True,
) as prof:

    for step in range(10):
        with record_function("train_step"):
            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                output = controller.traced_model(x)
                loss = criterion(output, y)
                loss.backward()

torch.cuda.synchronize()

print(prof.key_averages().table(
    sort_by="cuda_time_total",
    row_limit=200
))
